# Introduction

The notebook is intened to experiment with different technologies for the task of **Question and Answering**.

There are two different type of models:
- *Open Domain* - They do not require a passed context
- *Reading Comprehension* - They find the answer within a given context

Such models can work in two different approaches:
- *Open Book* - The model can access external source of information
- *Closed Book* - The model can only access what has been encoded in its paramters